In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker pull 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 578489032226.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker images

In [ ]:
!docker build -t 578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo -f Dockerfile.train .

In [ ]:
!docker images

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('file://{}/data/input/cancer_data.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')

In [ ]:
!tar -xzf data/output/output.tar.gz

In [ ]:
!tar -xzf ./data/output/model.tar.gz

In [ ]:
%%sh
# Specify an image name
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  578489032226.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker images

In [ ]:
!docker tag ciao:mondo 578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo

In [ ]:
!docker images

In [ ]:
!docker push 578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo',
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters,
    output_path='s3://itsrizzoli-flaviomanna/data/output/'
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('s3://itsrizzoli-flaviomanna/data/input/cancer_data.csv')
print('##### ESTIMATOR FIT COMPLETED')

In [ ]:
!pip3 install tensorflow pandas numpy scikit-learn keras

In [ ]:
!pip3 freeze > requirements.txt

In [ ]:
!FLASK_APP=${PWD}/inference.py flask run --host 0.0.0.0 --port 8080

In [ ]:
curl http://localhost:8080/ping

In [1]:
%%sh
# Specify an image name
image_name=tensorflow-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-inference ######################
account: 578489032226 ######################
region: us-east-1 ######################
fullname: 578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.0-cpu-py310-pluto-2 ######################



An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'tensorflow-inference' does not exist in the registry with id '578489032226'


{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:578489032226:repository/tensorflow-inference",
        "registryId": "578489032226",
        "repositoryName": "tensorflow-inference",
        "repositoryUri": "578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference",
        "createdAt": 1732290227.1,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker build -t 578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo -f Dockerfile.inference .

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 578489032226.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker push 578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo

In [ ]:
!SM_MODEL_DIR=${PWD} FLASK_APP=${PWD}/inference.py flask run --host 0.0.0.0 --port 8080

In [87]:
from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

my_session = boto3.session.Session()
aws_region = my_session.region_name

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

sagemaker_role = get_execution_role()

model_name = 'FM-AWS-first-try'
model_files = 's3://itsrizzoli-flaviomanna/data/output/tensorflow-training-2024-11-22-10-30-14-161/output/model.tar.gz'
model_image = '578489032226.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo'

In [88]:
# Create model
model_matches = sm_client.list_models(NameContains=model_name)["Models"]
if not model_matches:
    create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': model_image,
        'ModelDataUrl': model_files
    })
else:
    print(f"Model with name {model_name} already exists! Change model name to create new")


Model with name FM-AWS-first-try already exists! Change model name to create new


In [90]:

# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'la-mia-prima-api-inferenza-config'

instance_type = 'ml.p2.xlarge'


endpoint_config_matches = sm_client.list_endpoint_configs(NameContains=endpoint_config_name)["EndpointConfigs"]
if not endpoint_config_matches:
    endpoint_config_response = sagemaker_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        # You will specify this name in a CreateEndpoint request.
        # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
        ProductionVariants=[
            {
                "VariantName": "variant1", # The name of the production variant.
                "ModelName": model_name,
                "InstanceType": instance_type, # Specify the compute instance type.
                "InitialInstanceCount": 1 # Number of instances to launch initially.
            }
        ]
    )
    
    print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")
else:
    print(f"Endpoint config with name {endpoint_config_name} already exists! Change endpoint config name to create new")

Endpoint config with name la-mia-prima-api-inferenza-config already exists! Change endpoint config name to create new


In [91]:
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = f"{model_name}-endpoint"

endpoint_matches = sm_client.list_endpoints(NameContains=endpoint_name)["Endpoints"]
if not endpoint_matches:
    create_endpoint_response = sagemaker_client.create_endpoint(
        EndpointName=endpoint_name, 
        EndpointConfigName=endpoint_config_name
    )
    print(f"Created Endpoint with name: {endpoint_name}")

else:
    print(f"Endpoint with name {endpoint_name} already exists! Change endpoint name to create new")


In [93]:
import sagemaker
import time

sess = sagemaker.Session()
region = sess.boto_region_name
sm_client = boto3.client("sagemaker", region_name=region)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]

while status == "Creating":
    print(f"Endpoint Status: {status}...")
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
print(f"Endpoint Status: {status}")

Endpoint Status: Failed


In [ ]:
# Fetch test data to run predictions with the endpoint
test_df = pd.read_csv(test_data_uri)

# For content type text/csv, payload should be a string with commas separating the values for each feature
# This is the inference request serialization step
# CSV serialization
csv_file = io.StringIO()
test_sample = test_df.drop(["fraud"], axis=1).iloc[:5]
test_sample.to_csv(csv_file, sep=",", header=False, index=False)
payload = csv_file.getvalue()
response = sm_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType="text/csv",
    Accept="text/csv"
)

# This is the inference response deserialization step
# This is a bytes object
result = response["Body"].read()
# Decoding bytes to a string
result = result.decode("utf-8")
# Converting to list of predictions
result = re.split(",|\n",result)

prediction_df = pd.DataFrame()
prediction_df["Prediction"] = result[:5]
prediction_df["Label"] = test_df["fraud"].iloc[:5].values
prediction_df